In [1]:
import pandas as pd
from after_preprocess_function import split_strength
from after_preprocess_function import reorder_dfs
import re
import ast

In [2]:
in_col_mapper = {'Medicine Name':'product'}
map_ru_in = {"Intravenous" : "Injection"}

In [3]:
df_india = pd.read_csv('data_preprocessed/India_short.csv', index_col=0)

In [4]:
df_india = df_india.rename(columns=in_col_mapper)
df_india['route'] = df_india['route'].map(map_ru_in)
df_india['route'] = df_india['route'].str.upper()
df_india['product'] = df_india['product'].str.upper()
df_india['country'] = 'INDIA'
df_india = df_india.drop(columns=['Uses','Therapeutic Class'])
df_india = df_india.drop_duplicates()
df_india['strength'] = df_india['strength'].apply(lambda x: re.sub('\(|\)','',x))

df_india['drug'] = df_india['drug'].apply(lambda x: ast.literal_eval(x))
df_india['strength'] = df_india['strength'].apply(lambda x: str(x).split(','))

df_india = df_india.set_index(keys=['product','route','country']).apply(lambda x: x.apply(pd.Series).stack()).reset_index().drop('level_3',1)

df_india['drug'] = df_india['drug'].apply(lambda x: x.replace('+ ',''))

df_india[['strength','unit']] = pd.DataFrame(df_india['strength'].apply(split_strength).tolist(), index=df_india.index)
df_india = reorder_dfs(df_india)

In [5]:
df_india.to_csv('data_after_preprocess/data_india.csv', index=False)